In [1]:
import os

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data/country_image_keras'

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

img_width, img_height = 640, 640
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
num_classes = 5

/home/hidemaro/.pyenv/versions/3.6.5/envs/sound/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [3]:
train_data_dir = DATA_HOME_DIR + '/train'
validation_data_dir = DATA_HOME_DIR + '/validation'

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 317 images belonging to 5 classes.
Found 0 images belonging to 0 classes.


In [6]:
class Model():
    @staticmethod
    def buildConvModel():
        model = Sequential()
        
        model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
                  
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        
        return model
    
    @staticmethod
    def buildNeuralModel():
        model = Sequential()
        
        # input layer
        input_num = img_width * img_height * 3
        model.add(layers.Dense(10000, activation = "relu", input_dim=input_num))
        
        # hidden layers
        model.add(Dropout(0.3, noise_shape=None, seed=None))
        model.add(Dense(1000), activation='s igmoid')
        model.add(Dropout(0.2, noise_shape=None, seed=None))
        model.add(Dense(100), activation='sigmoid')
        
        #output layer
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        return model

In [7]:
model = Model().buildConvModel()
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 638, 638, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 638, 638, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 319, 319, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 317, 317, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 317, 317, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 158, 158, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 156, 156, 64)      18496     
__________

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Epoch 1/50
